In [1]:
# при запуске ноутбука рабочей директорией ставится не папка проекта, а папка examples, а в ней нет src, 
# соответственно импорты не работают. Эта строчка фиксит такое поведение для тестирование, у пользователей будет
# свой путь установки, в котором этой проблемы не будет. 
import sys
sys.path.insert(0, '..')

In [2]:
import pandas as pd

from src.eventstream.eventstream import Eventstream
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.graph.p_graph import PGraph, EventsNode

raw_data = pd.read_csv('simple-onlineshop.csv')

raw_data_schema = RawDataSchema(
    event_name="event", event_timestamp="timestamp", user_id="user_id")

source = Eventstream(
    raw_data=raw_data,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

graph = PGraph(source_stream=source)

In [4]:
# Add simple group
from src.data_processors_lib.simple_processors import SimpleGroup

cart_events = EventsNode(
    SimpleGroup(**{
        "event_name": "add_to_cart",
        "filter": lambda df, schema: df[schema.event_name].isin(["cart"]),
    })
)
graph.add_node(
    node=cart_events,
    parents=[graph.root]
)
# result = graph.combine(cart_events)
# result.to_dataframe().head()

TypeError: __init__() got an unexpected keyword argument 'event_name'

In [5]:
from src.data_processors_lib.simple_processors import StartEndEvents

start_end_events = EventsNode(
    StartEndEvents()
)

graph.add_node(
    node=start_end_events,
    parents=[cart_events]
)

result = graph.combine(
    node=start_end_events
)
result.to_dataframe().head(n=20)

/home/vladimir/Workspace/retentioneering/retentioneering-tools-new-arch/src/eventstream/eventstream.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.__events = result_left_part.append(
/home/vladimir/Workspace/retentioneering/retentioneering-tools-new-arch/src/eventstream/eventstream.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.__events = result_left_part.append(
/home/vladimir/Workspace/retentioneering/retentioneering-tools-new-arch/src/eventstream/eventstream.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.__events = result_left_part.append(
/home/vladimir/Workspace/retentioneering/retentioneering-tools-new-arch/src/eventstream/eventstream.py:216: FutureWarning: The frame.append method i

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,23a49677-bdbf-45f7-bf73-a5c19f29dc0d,start,0,start,2019-11-01 17:59:13.273932,219483890.0
2,eb4085d0-3239-42d3-8d40-5303d5c737ff,raw,2,product1,2019-11-01 17:59:28.459271,219483890.0
3,ba1feff9-0779-4684-ae8c-34f9bf85718d,group_alias,3,add_to_cart,2019-11-01 17:59:29.502214,219483890.0
5,6acb50c3-7467-4310-8cca-ba60a0784e00,raw,5,catalog,2019-11-01 17:59:32.557029,219483890.0
6,31b515f0-4732-45c9-9911-5a847805afb1,start,6,start,2019-11-01 21:38:19.283663,964964743.0
8,ed7db420-1052-43f7-a14b-3e74ac3ac2fc,group_alias,8,add_to_cart,2019-11-01 21:38:36.761221,964964743.0
10,d60fb190-c874-4d42-b59b-a6299d17fbf0,raw,10,delivery_choice,2019-11-01 21:38:37.564693,964964743.0
11,1be0b193-a601-40ba-923c-09ace1441828,start,11,start,2019-11-01 22:28:54.791683,629881394.0
12,8f2b2c0e-c24d-4e30-9b63-2a2ffcfff244,start,12,start,2019-11-01 22:28:54.791683,629881395.0
15,dcfd1e5a-fd8d-417e-bd90-7c120d485350,raw,15,catalog,2019-11-01 22:29:01.049513,629881394.0
